In [261]:
# This is Lecture 04a Exercise 1, 2, 3 (Missing Values in the Pima Indians Diabetes Database) 
# of the "Data Science" class at Technische Hochschule Rosenheim

## Missing Values in the "Pima Indians Diabetes Database"

We will practice out skills in detecting and handling missing values using a well known, real world dataset.

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

##### Dataset Source
Original Owners:
National Institute of Diabetes and Digestive and Kidney Diseases

Donor of database:
Vincent Sigillito (vgs '@' aplcen.apl.jhu.edu)Research Center, RMI Group LeaderApplied Physics LaboratoryThe Johns Hopkins UniversityJohns Hopkins RoadLaurel, MD 20707(301) 953-6231

Relevant Paper:
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care} (pp. 261*265). IEEE Computer Society Press.

### Attribute Information

**Number of Instances**: 768

**Number of Attributes**: 8 plus class 

1. Number of times pregnant 
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
3. Diastolic blood pressure (mm Hg) 
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml) 
6. Body mass index (weight in kg/(height in m)^2) 
7. Diabetes pedigree function (scores the likelihood of diabetes based on family history)
8. Age (years) 
9. Class variable (0 or 1)

### Missing Values in the Dataset

The donors of the dataset did not make any statements regarding missing values. However, there must be some: there are zeros in places where they are biologically impossible, such as the blood pressure attribute. It seems very likely that zero values encode missing data.

In [262]:
# imports
import numpy as np
import pandas as pd

In [263]:
# load the dataset
raw = pd.read_csv('data/pima-indians-diabetes.csv')
print(raw.dtypes)

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object


### Exercise 1
Explore the data and identify, in which attributes missing values may be present and how they are encoded. Clean the data by makeing sure that all missing values are encoded as `NaN`.

Hints: 
* Dataframes have a `describe()` method, which computes summary statistics and can be very usefulto explore. Note that `describe()` returns a Dataframe
* Transposing the result of the `describe()` methods makes it easier to read (imho). (with the `transpose()` method or the shortcut `T`)
* The `value_counts()` method (applicable to both Series and Dataframes) returns all values together with the information, how often they appear. This can be very useful for exploring attributes with low cardinality. It is even more useful if combined with the `sort_index()` method or the `sort_values()` method (which sort by index or values, as their name implies). For continuous attributes, the `bins` parameter of the `value_counts()` method is very helpful, it specifies the number of bins to group the data into (in my experience, 20 is a reasonable starting value).

In [264]:
# --------- SOLUTION
raw.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [265]:
# Pregnancies sus but possible. cap at 10-12? 0 can be wrong entries
# Glucose 0 impossible, low and high values seem plausible
# BloodPressure of 50 unhealthy, unlikely true. high is ok
# SkinThickness up to 40? 0 and 99 impossible
# Insulin 0 and 300+ sus
# BMI 0 impossible. Many fat people. 20-30 plausible
# DPF 2+ is a lot but can vary
# Age looks good

# raw.sort_values("DiabetesPedigreeFunction")["DiabetesPedigreeFunction"].head(60)
(raw == 0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

### Exercise 2
For each attribute containing missing values, decide on the assumption (MCAR, MAR, MNAR) to use and which approach to handling missing values you would use. Explain/justify your decisions!

In [266]:
# --------- SOLUTION
"""
Pregnancy values of 0 can be wrong. those cases would be MCAR or MNAR
-women just not wanting to disclose the number or bc they feel bad about the value
Glucose levels of 0 seem like MCAR
-lost data or not tested
BloodPressure of 0 seems like MCAR
-lost data or not tested
SkinThickness of 0 seems like MCAR or maybe MAR
-not able to test (maybe bc of the underlying condition)
Insulin 0 can be MCAR or MAR
-it is tested with Glucose levels but bad data?
BMI of 0 is MNAR
-not wanting to disclose because it's too high lol
"""
# raw = raw.replace(0,np.nan)
raw

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### Exercise 3
a) implement listwise deletion (complete-case analysis) for all attributes with missing values. How many rows do you have left?

b) implement simple imputation by the mean for all attributes with missing values

In [267]:
# --------- SOLUTION
clean = raw.copy()
boola = (raw[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]] == 0).any(axis=1)
clean = clean[boola].reset_index()
print(clean)

clean2 = raw.replace(0,np.nan) # 111x9
clean2 = clean2.fillna(clean2.mean())
print(clean2.mean())
clean2

     index  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0        0            6      148             72             35        0  33.6   
1        1            1       85             66             29        0  26.6   
2        2            8      183             64              0        0  23.3   
3        5            5      116             74              0        0  25.6   
4        7           10      115              0              0        0  35.3   
..     ...          ...      ...            ...            ...      ...   ...   
371    761            9      170             74             31        0  44.0   
372    762            9       89             62              0        0  22.5   
373    764            2      122             70             27        0  36.8   
374    766            1      126             60              0        0  30.1   
375    767            1       93             70             31        0  30.4   

     DiabetesPedigreeFuncti

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1.0
1,1.000000,85.0,66.0,29.00000,155.548223,26.6,0.351,31,1.0
2,8.000000,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1.0
3,1.000000,89.0,66.0,23.00000,94.000000,28.1,0.167,21,1.0
4,4.494673,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1.0
...,...,...,...,...,...,...,...,...,...
763,10.000000,101.0,76.0,48.00000,180.000000,32.9,0.171,63,1.0
764,2.000000,122.0,70.0,27.00000,155.548223,36.8,0.340,27,1.0
765,5.000000,121.0,72.0,23.00000,112.000000,26.2,0.245,30,1.0
766,1.000000,126.0,60.0,29.15342,155.548223,30.1,0.349,47,1.0


---